In [1]:
from connect import *
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm; tqdm.pandas()

In [115]:
number_of_events = connect().get_user_owned_events(user_id())['pagination']['object_count']
print('Total number of events:', number_of_events)

Total number of events: 12


In [3]:
total_rsvps = connect().get_user_owned_event_orders(user_id())['pagination']['object_count']
print(f'Total number of RSVPs:', total_rsvps)

Total number of RSVPs: 759


In [4]:
page_count = connect().get_user_owned_event_orders(user_id())['pagination']['page_count']
print(f'Total number of contact pages:', page_count)

Total number of contact pages: 16


#### YERR

In [5]:
class CaseInsensitively(object):
    def __init__(self, s):
        self.__s = s.lower()
    def __hash__(self):
        return hash(self.__s)
    def __eq__(self, other):
        # ensure proper comparison between instances of this class
        try:
           other = other.__s
        except (TypeError, AttributeError):
          try:
             other = other.lower()
          except:
             pass
        return self.__s == other

In [199]:
def get_contact_list(number_of_pages):
    return [
        connect().get_user_owned_event_orders(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Contact Pages to Scrape")
    ]

In [200]:
def convert_contact_list_to_dataframe(contact_list):
    df = pd.DataFrame()
    for i in range(0, len(contact_list)):
        df = df.append(contact_list[i]['orders'])
    
    return df

In [201]:
data_frame = convert_contact_list_to_dataframe(get_contact_list(page_count))

In [243]:
data_frame['full_name'] = data_frame.name

# EVENTS

In [166]:
event_page_count = connect().get_user_owned_events(user_id())['pagination']['page_count']
print(f'Total number of Event pages:', event_page_count)

Total number of Event pages: 1


In [167]:
def get_event_list(number_of_pages):
    return [
        connect().get_user_owned_events(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Event Pages to Scrape")
    ]

In [168]:
def convert_event_list_to_dataframe(event_list):
    df = pd.DataFrame()
    for i in range(0, len(event_list)):
        df = df.append(event_list[i]['events'])
    
    return df

In [244]:
event_list = get_event_list(event_page_count)

In [245]:
def create_event_name_column(event_list):
    for i in range(0, len(event_list)):
        for event in range(0, number_of_events):
            event_list[i]['events'][event]['event_name'] = event_list[i]['events'][event]['name']['text']
    
    return event_list

In [246]:
event_data_frame = convert_event_list_to_dataframe(create_event_name_column(event_list))

In [247]:
event_data_frame

,capacity,capacity_is_custom,category_id,changed,created,currency,description,end,event_name,format_id,...,source,start,status,subcategory_id,summary,tx_time_limit,url,vanity_url,venue_id,version
0,1000,False,101,2018-09-20T04:36:03Z,2018-08-18T18:09:18Z,USD,{'text': 'Please join us as we introduce Black...,"{'timezone': 'America/New_York', 'local': '201...",Black Tech Columbus - Kick Off Event!,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,1999,Please join us as we introduce Black Tech Colu...,480,https://www.eventbrite.com/e/black-tech-columb...,https://blacktechcolumbus.eventbrite.com,26282186,3.0.0
1,75,False,199,2018-11-02T04:12:56Z,2018-10-16T01:44:03Z,USD,{'text': 'Join us as we share our mission for ...,"{'timezone': 'America/New_York', 'local': '201...",Welcome to #BlackTech614!,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,None,Join us as we share our mission for #BlackTech...,480,https://www.eventbrite.com/e/welcome-to-blackt...,https://blacktech614.eventbrite.com,27391690,3.0.0
2,200,False,116,2018-12-10T11:40:24Z,2018-11-21T15:45:28Z,USD,{'text': 'Join #BlackTech614 for Christmas in ...,"{'timezone': 'America/New_York', 'local': '201...",Christmas in Wakanda,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,16006,Join #BlackTech614 for Christmas in Wakanda on...,480,https://www.eventbrite.com/e/christmas-in-waka...,https://christmasinwakanda.eventbrite.com,28030990,3.0.0
3,60,False,102,2019-01-16T05:18:10Z,2018-12-27T18:32:15Z,USD,{'text': 'Join #BlackTech614 as we present An ...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 Presents: An Intro to BLOCKCHAIN,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join #BlackTech614 as we present An Intro to B...,480,https://www.eventbrite.com/e/blacktech614-pres...,https://blacktech614blockchain.eventbrite.com,28475539,3.0.0
4,85,False,102,2019-02-08T05:15:03Z,2019-01-22T23:21:51Z,USD,{'text': 'Join us as we kick off our 2019 Lear...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 Introduces: 2019 Learning Commun...,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join us as we kick off our 2019 Learning Commu...,480,https://www.eventbrite.com/e/blacktech614-intr...,https://bt614lcs.eventbrite.com,28976312,3.0.0
5,250,False,102,2019-02-22T05:02:58Z,2019-01-25T00:31:20Z,USD,{'text': 'Please join us as we collaborate wit...,"{'timezone': 'America/New_York', 'local': '201...",The Power of Technology in Achieving Your Dive...,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Please join us as we collaborate with the Wend...,480,https://www.eventbrite.com/e/the-power-of-tech...,https://bt614xwendys.eventbrite.com,29388764,3.0.0
6,85,False,102,2019-03-08T08:50:31Z,2019-02-23T16:41:18Z,USD,{'text': 'Please join us for our first Learnin...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 - Data 101 Learning Community,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Please join us for our first Learning Communit...,480,https://www.eventbrite.com/e/blacktech614-data...,https://data101.eventbrite.com,29977638,3.0.0
7,85,False,102,2019-03-12T03:58:11Z,2019-02-20T18:14:56Z,USD,{'text': 'Join us as we kickoff our 3 committe...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 Committee Kickoff!,10,...,create_2.0,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join us as we kickoff our 3 committees for 201...,480,https://www.eventbrite.com/e/blacktech614-comm...,https://bt614committees.eventbrite.com,30217499,3.0.0
8,1000,False,102,2019-03-21T07:44:28Z,2019-03-10T19:40:53Z,USD,{'text': 'Please join us as we network with Bl...,"{'timezone': 'America/New_York', 'local': '201...",Happy Hour - Ease into Spring with #BlackTech614!,10,...,create_2.0,"{'timezone': 'Ame

# CONTACTS + EVENTS

In [248]:
data_frame.head()

,changed,costs,created,email,event_id,first_name,id,last_name,name,resource_uri,status,time_remaining,full_name
0,2018-08-28T15:11:38Z,"{'base_price': {'display': '$0.00', 'currency'...",2018-08-28T15:11:37Z,okeydopeness@gmail.com,49248081337,Jenna,814868885,El-Amin,Jenna El-Amin,https://www.eventbriteapi.com/v3/orders/814868...,placed,None,Jenna El-Amin
1,2018-08-28T16:19:51Z,"{'base_price': {'display': '$0.00', 'currency'...",2018-08-28T16:19:51Z,d.hodge@getwiredlabs.com,49248081337,Derrick,814909326,Hodge,Derrick Hodge,https://www.eventbriteapi.com/v3/orders/814909...,placed,None,Derrick Hodge
2,2018-08-28T16:34:19Z,"{'base_price': {'display': '$0.00', 'currency'...",2018-08-28T16:34:19Z,dbarnett@increasecdc.org,49248081337,Deonna,814917294,Barnett,Deonna Barnett,https://www.eventbriteapi.com/v3/orders/814917...,placed,None,Deonna Barnett
3,2018-08-28T16:59:06Z,"{'base_price': {'display': '$0.00', 'currency'...",2018-08-28T16:59:05Z,lrturn@gmail.com,49248081337,Lisa,814930893,Turner,Lisa Turner,https://www.eventbriteapi.com/v3/orders/814930...,placed,None,Lisa Turner
4,2018-08-28T17:03:37Z,"{'base_price': {'display': '$0.00', 'currency'...",2018-08-28T17:03:37Z,marcelladmccurdy@gmail.com,49248081337,Dr. Marcella,814933878,McCurdy,Dr. Marcella McCurdy,https://www.eventbriteapi.com/v3/orders/814933...,placed,None,Dr. Marcella McCurdy


In [249]:
event_data_frame['event_id'] = event_data_frame.id

In [250]:
event_data_frame.head()

,capacity,capacity_is_custom,category_id,changed,created,currency,description,end,event_name,format_id,...,start,status,subcategory_id,summary,tx_time_limit,url,vanity_url,venue_id,version,event_id
0,1000,False,101,2018-09-20T04:36:03Z,2018-08-18T18:09:18Z,USD,{'text': 'Please join us as we introduce Black...,"{'timezone': 'America/New_York', 'local': '201...",Black Tech Columbus - Kick Off Event!,10,...,"{'timezone': 'America/New_York', 'local': '201...",completed,1999,Please join us as we introduce Black Tech Colu...,480,https://www.eventbrite.com/e/black-tech-columb...,https://blacktechcolumbus.eventbrite.com,26282186,3.0.0,49248081337
1,75,False,199,2018-11-02T04:12:56Z,2018-10-16T01:44:03Z,USD,{'text': 'Join us as we share our mission for ...,"{'timezone': 'America/New_York', 'local': '201...",Welcome to #BlackTech614!,10,...,"{'timezone': 'America/New_York', 'local': '201...",completed,None,Join us as we share our mission for #BlackTech...,480,https://www.eventbrite.com/e/welcome-to-blackt...,https://blacktech614.eventbrite.com,27391690,3.0.0,51444730574
2,200,False,116,2018-12-10T11:40:24Z,2018-11-21T15:45:28Z,USD,{'text': 'Join #BlackTech614 for Christmas in ...,"{'timezone': 'America/New_York', 'local': '201...",Christmas in Wakanda,10,...,"{'timezone': 'America/New_York', 'local': '201...",completed,16006,Join #BlackTech614 for Christmas in Wakanda on...,480,https://www.eventbrite.com/e/christmas-in-waka...,https://christmasinwakanda.eventbrite.com,28030990,3.0.0,52910420492
3,60,False,102,2019-01-16T05:18:10Z,2018-12-27T18:32:15Z,USD,{'text': 'Join #BlackTech614 as we present An ...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 Presents: An Intro to BLOCKCHAIN,10,...,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join #BlackTech614 as we present An Intro to B...,480,https://www.eventbrite.com/e/blacktech614-pres...,https://blacktech614blockchain.eventbrite.com,28475539,3.0.0,54143311101
4,85,False,102,2019-02-08T05:15:03Z,2019-01-22T23:21:51Z,USD,{'text': 'Join us as we kick off our 2019 Lear...,"{'timezone': 'America/New_York', 'local': '201...",#BlackTech614 Introduces: 2019 Learning Commun...,10,...,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join us as we kick off our 2019 Learning Commu...,480,https://www.eventbrite.com/e/blacktech614-intr...,https://bt614lcs.eventbrite.com,28976312,3.0.0,55250505750


In [251]:
def combined_contacts_and_events(contacts, events):
    return pd.concat(
        [
            contacts.set_index('event_id'),
            events.set_index('event_id')
        ], axis=1, join='inner'
    ).reset_index()

In [252]:
black_tech_614 = combined_contacts_and_events(data_frame, event_data_frame)

In [253]:
black_tech_614.groupby(['full_name', 'email'])['event_name'].count().sort_values(ascending=False)

full_name             email                             
Samuel Davis          samuel.davis@das.ohio.gov             10
Daron McDonald        daron06@yahoo.com                     10
Aaron Grady           aaron.grady15@gmail.com                8
Omar Cochran          Omar.J.Cochran@gmail.com               7
Bryan Goolsby         bryanjgoolsby@gmail.com                6
Craig Chavis          cchavis@chaveconsulting.com            6
Ashley Johnson        ashleylevjohnson@gmail.com             6
Craig Chavis Jr       craigmchavisjr@gmail.com               6
Leila Head            leilahead2@gmail.com                   5
Evian Napier          eviannapier@gmail.com                  5
Myronn Carlock        sigmaphine1914@gmail.com               5
Craig Collins         gomab86@gmail.com                      4
Kayondia Bradley      kayondiab@yahoo.com                    4
DAVID ADEYANJU        davidyanju@live.com                    4
Rufus Smith           rsmith@greenkeyllc.com                 

In [ ]:
# contact_list = {}

# for page_number in tqdm(range(0, page_count), total=page_count, desc="Contact Pages to Scrape"):
#     for attendee_index in range(0, 51):
#         contact_name = attendee_info(page_number, attendee_index=attendee_index).strip().lower()
        
#         if CaseInsensitively(contact_name) not in list(contact_list.keys()):
#             contact_list[contact_name] = {item: attendee_info(page_number, attendee_index=attendee_index, info=item)
#                                           for item in ['first_name', 'last_name', 'email']}
#             contact_list[contact_name]['total_rsvps'] = 1
#         else:
#             contact_list[contact_name]['total_rsvps'] += 1

In [ ]:
# black_tech_614_rsvps.head()#.sort_values(by=['total_rsvps'], ascending=False)

In [43]:
# pd.read_csv('black_tech_614_rsvps.csv')

In [763]:
def get_eventbrite_list(list_type='contact'):
    return get_contact_list() if list_type == 'contact' else create_event_name_column(get_event_list())


def get_contact_list():
    number_of_pages = connect().get_user_owned_event_orders(user_id())['pagination']['page_count']
    
    return [
        connect().get_user_owned_event_orders(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Contact Pages to Scrape")
    ]


def create_event_name_column(event_list):
    for i in range(0, len(event_list)):
        for event in range(0, number_of_events):
            event_list[i]['events'][event]['event_name'] = event_list[i]['events'][event]['name']['text']
    
    return event_list


def get_event_list():
    number_of_pages = connect().get_user_owned_events(user_id())['pagination']['page_count']
    
    return [
        connect().get_user_owned_events(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Event Pages to Scrape")
    ]

In [764]:
def convert_eventbrite_list_to_dataframe(eventbrite_list, list_type='contact'):
    df = pd.DataFrame()
    
    return convert_contact_list_to_dataframe(eventbrite_list, df) if list_type == 'contact'\
           else convert_event_list_to_dataframe(eventbrite_list, df)


def convert_contact_list_to_dataframe(contact_list, df):
    for i in range(0, len(contact_list)):
        df = df.append(contact_list[i]['orders'])
    return df.rename(columns={'name': 'full_name', 'id': 'contact_id'}).reset_index(drop=True)
    
    
def convert_event_list_to_dataframe(event_list, df):
    for i in range(0, len(event_list)):
        df = df.append(event_list[i]['events'])
    return df.rename(columns={'id': 'event_id'}).reset_index(drop=True)

In [765]:
def combined_contacts_and_events(contacts_dataframe, events_dataframe):
    return pd.concat(
        [
            contacts_dataframe.set_index('event_id'),
            events_dataframe.set_index('event_id')
        ], axis=1, join='inner'
    ).reset_index()

In [766]:
def lowercase_and_strip(df, columns=['full_name', 'first_name', 'last_name', 'email']):
    for column in columns:
        df[column] = df[column].str.lower().str.strip()

    return replace_delimeters(df)


def replace_delimeters(df, columns=['full_name', 'first_name', 'last_name']):
    for column in columns:
        df[column] = df[column].str.lower().str.strip().str.replace('[^\w\s]','')

    return df

In [767]:
contacts = convert_eventbrite_list_to_dataframe(get_eventbrite_list())
events = convert_eventbrite_list_to_dataframe(get_eventbrite_list(list_type='event'), list_type='event')

In [768]:
contacts_and_events = lowercase_and_strip(combined_contacts_and_events(contacts, events))

# FUZZY MATCH SHIT

In [769]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [770]:
unique_names = contacts_and_events['full_name'].unique()

possibilities = []
for name in unique_names:
    possibilities.append(process.extract(name, unique_names, limit=15, scorer=fuzz.token_sort_ratio))

In [771]:
for i in range(0, len(possibilities)):
    matching_name = possibilities[i][0][0]
    possible_name = possibilities[i][1][0]
    
    if contacts_and_events[(contacts_and_events.full_name == matching_name) | (contacts_and_events.full_name == possible_name)].email.nunique() == 1:
        contacts_and_events.loc[
            (contacts_and_events.full_name == matching_name) |
            (contacts_and_events.full_name == possible_name), 'full_name'] = matching_name
    elif contacts_and_events[(contacts_and_events.full_name == matching_name) | (contacts_and_events.full_name == possible_name)].email.nunique() > 1:
        contacts_and_events.loc[
            contacts_and_events.apply(lambda row: get_ratio(row), axis=1) > 90,
            'full_name'
        ] = matching_name

In [821]:
def get_ratio(row):
    name = row.full_name
    possible_name = row.possible_match
    
    return fuzz.token_sort_ratio(name, possible_name)

contacts_and_events.loc[
    contacts_and_events.apply(lambda row: get_ratio(row), axis=1) > 90,
    'full_name'
]

,event_id,changed,costs,created,email,first_name,contact_id,last_name,full_name,resource_uri,...,start,status,subcategory_id,summary,tx_time_limit,url,vanity_url,venue_id,version,possible_match
339,55250505750,2019-02-04T00:19:02Z,"{'base_price': {'display': '$0.00', 'currency'...",2019-02-04T00:19:02Z,lbaker@covermymeds.com,lachandra,893383155,baker,lachandra baker,https://www.eventbriteapi.com/v3/orders/893383...,...,"{'timezone': 'America/New_York', 'local': '201...",completed,2999,Join us as we kick off our 2019 Learning Commu...,480,https://www.eventbrite.com/e/blacktech614-intr...,https://bt614lcs.eventbrite.com,28976312,3.0.0,lachandra b baker
714,59293445291,2019-04-09T14:52:45Z,"{'base_price': {'display': '$0.00', 'currency'...",2019-04-09T14:52:44Z,lachandrabbaker@gmail.com,lachandra b,931171715,baker,lachandra b baker,https://www.eventbriteapi.com/v3/orders/931171...,...,"{'timezone': 'America/New_York', 'local': '201...",completed,None,Please join us for a discussion around Black T...,480,https://www.eventbrite.com/e/blacktech614-pres...,https://blacktechpanel.eventbrite.com,31284451,3.0.0,lachandra baker


**Unique Contacts**

In [724]:
contacts_and_events.full_name.nunique()

431

**Unique Emails**

In [722]:
contacts_and_events.email.nunique()

468